In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from openai import OpenAI
import subprocess

class State(TypedDict):
    video_file: str
    audio_file: str
    transcription: str

In [2]:
def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4", "mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file,
    ]

    subprocess.run(command)
    return {"audio_file": output_file}


def transcribe_audio(state: State):
    # audio_file = state["audio_file"]
    client = OpenAI()
    with open(state["audio_file"], mode="rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            language="ko",
            prompt="Netherlands, Rotterdam, Amsterdam, The Hage",
        )
    return {
        "transcription": transcription,
    }

In [3]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_edge("transcribe_audio", END)

graph = graph_builder.compile()

In [4]:
graph.invoke(
    {
        "video_file": "./video.mp4"
    }
)

{'video_file': './video.mp4',
 'audio_file': './video.mp3',
 'transcription': '자소서가 언덕듣기에는 쓰기 쉬울 것 같아요 근데 여러분도 써보셔서 아시겠지만 가장 어려운 부분이 또 자소서이지 않나 자소서 쓸 때 하지 말아야 될 것부터 짚고 넘어가면 좋을 것 같아요 지금 보실 때 진짜 하지 말아야 되는 거 탈락하는 자소서는 뭐가 있을까요? 빠밤 빠밤 제가 계속 따라해야 되나요? 저희 시니어 면접위원분들께서 하시는 말씀 있는데 나는 딱 보면 알아 나름 말씀 많이 하세요 너무하신 거 아니에요? 맞습니다 그래서 그 직감에 의한 평가는 하지 않도록 저희가 계속 컨트롤을 하는 거고요 그래서 면접위원도 교육을 하는 건데요 그런데 자기소개서는 정말 딱 보면 알 수 있는 방법이 있어요 이건 직감이 아니고요 여러분들이 쓰신 자기소개서의 구조를 보면 딱 알 수 있어요 어떤 분들은 기승 경별로 막 앞쪽에서 막 이야기를 하세요 스토리텔링 하시다가 맨 마지막에 하고 싶은 말 이렇게 쓰시는데 그렇게 쓰시면 안 돼요 여러분 항상 앞에 성과, 결과부터 쓰시고 그다음에 그 성과 결과를 만들어낸 나의 액션, 상황 이런 것들을 쓰시는 거예요 시간이 없어 혹은 분량이 너무 쏟아져 라고 하면 성과만 쓰셔도 돼요 성과 플러스 나의 액션만 쓰셔도 돼요 음 성과 플러스 액션만 써야 돼요 다른 거는 안 쓰셔도 돼요 그거는 면접 때 여러분들에게 어차피 여쭤볼 거예요 그러니까 성과, 나의 액션 이런 생각을 해주셔야 되고요 두 번째 여러분들 너무 안타까운 게 있어요 내가 너무 잘났다라는 것을 보여주고 싶고 내가 이렇게 어려운 것을 했어라 하는 것을 보여주고 싶으신 건 알겠는데 이게 과연 지원하시는 회사의 직무와 어떤 연관성이 있는지를 꼭 확인을 하셔야 돼요 그것을 어필을 하셔야 돼요 그런 부분으로 해서 내가 원하는, 나의 달란 모습이 아니라 회사에서 나를 뽑고 싶어 하는지 그거를 타깃으로 해서 글을 쓰신다고 한다면 분명히 매력적인 자기소개서를 쓰실 수